In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-11-15 15:44:36.068101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763221476.366278      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763221476.457704      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset_path = "/kaggle/input/brain-tumor-mri-dataset/"
os.listdir(dataset_path)

['Training', 'Testing']

In [3]:
# DImensiones de las imagenes 
img_width, img_height = 150, 150

# Definir rutas especificas
train_dir = os.path.join(dataset_path, "Training")
test_dir = os.path.join(dataset_path, "Testing")


In [4]:
# Aumentación de datos

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [5]:
# CREAR EL MODELO CNN

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(), 
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') #Sigmoid porque solo revisa 2: Tumor o no tumor
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-15 15:44:56.043708: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [7]:
# Entrenamiento del modelo
history = model.fit(
    train_generator, 
    epochs=15, 
    validation_data=test_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 175s 960ms/step - accuracy: 0.2386 - loss: -132895824.0000 - val_accuracy: 0.2334 - val_loss: -7343606272.0000
Epoch 2/15
 42/179 ━━━━━━━━━━━━━━━━━━━━ 1:56 852ms/step - accuracy: 0.2059 - loss: -10450477056.0000

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'], label='Precision Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precision Validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)
plt.show()

plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida Validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Perdida')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Predicción de Una Sola Imagen

test_image_path = os.path.join(test_dir, 'notumor', 'Te-no_0010.jpg')

# Cargar Imagen
img = cv2.imread(test_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (img_width, img_height))

img_normalized = img_resized / 255.0
img_input = np.expand_dims(img_normalized, axis=0)

prediction = model.predict(img_input)[0][0]

if prediction > 0.5:
    resultado = "Tumor detectado"
    color = "red"
else: 
    resultado = "No se detecto tumor"
    color = "green"

plt.figure(figsize=(5,5))
plt.imshow(img_resized)
plt.title(f"{resultado} ({prediction:.2f})", color = color)
plt.axis('off')
plt.show()